<a href="https://colab.research.google.com/github/Ndthain25092002/Intent-detection-and-slot-filling/blob/main/ID_SF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchtext==0.6.0

In [ ]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import math
import time
import os
from torchtext import data, datasets
import pickle
import torch.optim as optim
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
import csv

In [ ]:
# Sử dụng hàm lambda để định nghĩa phương thức tách từ đơn giản, ở đây là tách theo khoảng trắng
tokenize = lambda s: s.split()

# Định nghĩa trường TEXT (dùng để xử lý văn bản)
TEXT = data.Field(
    sequential=True,         # Dữ liệu là chuỗi văn bản
    tokenize=tokenize,       # Phương thức tách từ sử dụng hàm lambda trên
    lower=True,               # Chuyển đổi văn bản thành chữ thường
    use_vocab=True,           # Sử dụng từ điển
    init_token='<sos>',       # Thêm token bắt đầu chuỗi
    eos_token='<eos>',        # Thêm token kết thúc chuỗi
    pad_token='<pad>',        # Thêm token dùng để đệm độ dài câu
    unk_token='<unk>',        # Thêm token cho từ không xác định
    batch_first=True,         # Batch đầu tiên trong tensor có kích thước (batch_size, seq_len, ...)
    fix_length=50,            # Đặt độ dài cố định cho mỗi chuỗi
    include_lengths=True      # Bao gồm thông tin chiều dài của mỗi chuỗi trong kết quả
)

# Định nghĩa trường SLOT (tương tự như TEXT)
SLOT = data.Field(
    sequential=True,
    tokenize=tokenize,
    lower=True,
    use_vocab=True,
    init_token='<sos>',
    eos_token='<eos>',
    pad_token='<pad>',
    unk_token='<unk>',
    batch_first=True,
    fix_length=50,
    include_lengths=True
)

# Định nghĩa trường INTENT (tương tự như TEXT, nhưng không phải là chuỗi mà là một giá trị duy nhất)
INTENT = data.Field(
    sequential=False,  # Không phải là chuỗi
    use_vocab=True
)

# Tạo tập dữ liệu huấn luyện và tập dữ liệu validation từ các file CSV
train, val = data.TabularDataset.splits(
    path='./',
    skip_header=True,
    train='train.csv',
    validation='val.csv',
    format='csv',
    fields=[('text', TEXT), ('slot', SLOT), ('intent', INTENT)]
)

# Xây dựng từ điển từ vựng cho TEXT, SLOT và INTENT từ tập dữ liệu huấn luyện và validation
TEXT.build_vocab(train, val)
SLOT.build_vocab(train, val)
INTENT.build_vocab(train, val)

# Tạo các iterators cho tập huấn luyện và tập validation
train_iter, val_iter = data.Iterator.splits(
    (train, val),
    batch_sizes=(32, len(val)),
    shuffle=True,
    sort_within_batch=True,
    sort_key=lambda x: len(x.text)
)

# Tạo tập dữ liệu kiểm tra từ file CSV
test = data.TabularDataset(
    path='test.csv',
    format='csv',
    fields=[('text', TEXT), ('slot', SLOT), ('intent', INTENT)],
    skip_header=True
)

# Tạo iterator cho tập dữ liệu kiểm tra
test_iter = data.Iterator(
    test,
    batch_size=len(test),
    shuffle=True,
    sort_within_batch=True,
    sort_key=lambda x: len(x.text)
)


In [ ]:
# save text
text_words_path = os.path.join(os.getcwd(), 'text_words.pkl')
with open(text_words_path, 'wb') as f_text_words:
    pickle.dump(TEXT.vocab, f_text_words)

# save slot
slot_words_path = os.path.join(os.getcwd(), 'slot_words.pkl')
with open(slot_words_path, 'wb') as f_slot_words:
    pickle.dump(SLOT.vocab, f_slot_words)

# save intent
intent_words_path = os.path.join(os.getcwd(), 'intent_words.pkl')
with open(intent_words_path, 'wb') as f_intent_words:
    pickle.dump(INTENT.vocab, f_intent_words)

TEXT.vocab.stoi[TEXT.pad_token]


1

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Định nghĩa một mô hình PyTorch Module cho SlotGate
class SlotGate(nn.Module):
    def __init__(self, hidden_dim):
        super(SlotGate, self).__init__()
        # Tạo các lớp tuyến tính cho SlotGate
        self.fc_intent_context = nn.Linear(hidden_dim, hidden_dim)
        self.fc_v = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, slot_context, intent_context):
        # Tuyến tính hóa intent_context
        intent_context_linear = self.fc_intent_context(intent_context)

        # Tổng của slot_context và intent_context sau khi được biến đổi tuyến tính
        sum_intent_slot_context = slot_context + intent_context_linear

        # Hàm kích hoạt tanh theo phần tử sau đó làm biến đổi tuyến tính khác
        fc_linear = self.fc_v(torch.tanh(sum_intent_slot_context))

        # Tính tổng theo chiều của chuỗi
        sum_gate_vec = torch.sum(fc_linear, dim=1)
        return sum_gate_vec

# Định nghĩa một mô hình PyTorch Module cho AttnContext
class AttnContext(nn.Module):
    def __init__(self, hidden_dim):
        super(AttnContext, self).__init__()

    def forward(self, hidden, text_output_hidden):
        # Thêm chiều của chuỗi cho 'hidden'
        hidden = hidden.unsqueeze(1)

        # Tính trọng số chú ý dựa trên tích phần tử
        attn_weight = torch.sum(hidden * text_output_hidden, dim=2)

        # Áp dụng softmax theo chiều của chuỗi
        attn_weight = F.softmax(attn_weight, dim=1).unsqueeze(1)

        # Thực hiện tổng có trọng số của chú ý
        attn_vector = attn_weight.bmm(text_output_hidden)

        # Squeeze chiều của chuỗi
        return attn_vector.squeeze(1)

In [ ]:
# Định nghĩa một mô hình PyTorch Module cho Bi-directional LSTM với Attention và SlotGate
class BirnnAttentionGate(nn.Module):
    def __init__(self, text_input_dim, text_emb_dim, hidden_dim, n_layers, dropout, pad_index, slot_output_size, intent_output_size, seq_len, predict_flag, slot_attention_flag):
        super(BirnnAttentionGate, self).__init__()

        # Các tham số của mô hình
        self.pad_index = pad_index
        self.hidden_dim = hidden_dim // 2  # Vì sử dụng LSTM bidirectional
        self.n_layers = n_layers
        self.slot_output_size = slot_output_size
        self.predict_flag = predict_flag
        self.slot_attention_flag = slot_attention_flag

        # Embedding layer cho văn bản đầu vào
        self.text_embedding = nn.Embedding(text_input_dim, text_emb_dim, padding_idx=pad_index)

        # LSTM bidirectional cho việc biểu diễn văn bản
        self.text_lstm = nn.LSTM(text_emb_dim, self.hidden_dim, n_layers, dropout=dropout, bidirectional=True, batch_first=True)

        # Các thành phần cho Attention và SlotGate
        self.slot_context = AttnContext(hidden_dim)
        self.intent_context = AttnContext(hidden_dim)
        self.slotGate = SlotGate(hidden_dim)

        # Linear layers cho đầu ra dự đoán
        self.intent_output = nn.Linear(hidden_dim, intent_output_size)
        self.slot_output = nn.Linear(hidden_dim, slot_output_size)

    def forward(self, text_input, text_len):
        # Kiểm tra xem đang trong quá trình dự đoán hay không
        if self.predict_flag:
            assert len(text_input) == 1, 'Predicting one sentence at a time'
            seq_len = text_len[0]

            # Embedding văn bản đầu vào
            text_embedded = self.text_embedding(text_input)

            # Pack và unpack văn bản sử dụng PackedSequence để xử lý với độ dài biến đổi
            packed = torch.nn.utils.rnn.pack_padded_sequence(text_embedded, text_len, batch_first=True, enforce_sorted=True)
            text_output, (hidden, cell) = self.text_lstm(packed)
            text_output, _ = torch.nn.utils.rnn.pad_packed_sequence(text_output, batch_first=True, padding_value=self.pad_index, total_length=len(text_input[0]))

            # Khởi tạo tensor đầu ra cho slot predictions
            batch_size = text_input.shape[0]
            seq_len = text_input.shape[1]
            slot_outputs = torch.zeros(batch_size, seq_len, self.slot_output_size).to(device)
            aligns = text_output.transpose(0, 1)

            # Lặp qua từng thời điểm trong chuỗi
            for t in range(seq_len):
                aligned = aligns[t]

                # Tính toán slot context và intent context
                if self.slot_attention_flag:
                    slot_context = self.slot_context(aligned, text_output)
                    intent_context = self.intent_context(text_output[:, -1, :], text_output)
                    slot_gate = self.slotGate(slot_context, intent_context)
                    slot_gate = slot_gate.unsqueeze(1)
                    slot_context_gate = slot_gate * slot_context
                else:
                    intent_context = self.intent_context(text_output[:, -1, :], text_output)
                    slot_gate = self.slotGate(text_output[:, t, :], intent_context)
                    slot_gate = slot_gate.unsqueeze(1)
                    slot_context_gate = slot_gate * text_output[:, t, :]

                # Dự đoán slot và lưu vào tensor đầu ra
                slot_prediction = self.slot_output(slot_context_gate + text_output[:, t, :])
                slot_outputs[:, t, :] = slot_prediction

            # Dự đoán intent từ context cuối cùng
            intent_outputs = self.intent_output(intent_context + text_output[:, -1, :])

            return slot_outputs, intent_outputs

        else:
            # Đoạn code tương tự như ở trên, nhưng không có quá trình dự đoán
            text_embedded = self.text_embedding(text_input)
            packed = torch.nn.utils.rnn.pack_padded_sequence(text_embedded, text_len, batch_first=True, enforce_sorted=True)
            text_output, (hidden, cell) = self.text_lstm(packed)
            text_output, _ = torch.nn.utils.rnn.pad_packed_sequence(text_output, batch_first=True, padding_value=self.pad_index, total_length=len(text_input[0]))

            batch_size = text_input.shape[0]
            seq_len = text_input.shape[1]
            slot_outputs = torch.zeros(batch_size, seq_len, self.slot_output_size).to(device)
            aligns = text_output.transpose(0, 1)

            for t in range(seq_len):
                aligned = aligns[t]

                if self.slot_attention_flag:
                    slot_context = self.slot_context(aligned, text_output)
                    intent_context = self.intent_context(text_output[:, -1, :], text_output)
                    slot_gate = self.slotGate(slot_context, intent_context)
                    slot_gate = slot_gate.unsqueeze(1)
                    slot_context_gate = slot_gate * slot_context
                else:
                    intent_context = self.intent_context(text_output[:, -1, :], text_output)
                    slot_gate = self.slotGate(text_output[:, t, :], intent_context)
                    slot_gate = slot_gate.unsqueeze(1)
                    slot_context_gate = slot_gate * text_output[:, t, :]

                slot_prediction = self.slot_output(slot_context_gate + text_output[:, t, :])
                slot_outputs[:, t, :] = slot_prediction

            intent_outputs = self.intent_output(intent_context + text_output[:, -1, :])

            return slot_outputs, intent_outputs

In [ ]:
def build_model(text, slot, intent, text_emb_dim, hidden_dim, n_layers, dropout, lr, gamma, weight_decay, seq_len):

    # Xác định kích thước đầu vào, đầu ra và số lớp nhãn
    input_dim = len(text.vocab)
    output_dim = len(slot.vocab)
    label_dim = len(intent.vocab)

    # Xây dựng mô hình BirnnAttentionGate
    model = BirnnAttentionGate(input_dim, text_emb_dim, hidden_dim, n_layers, dropout, text.vocab.stoi[text.pad_token], output_dim, label_dim, seq_len, False, True).to(device)

    # Áp dụng hàm khởi tạo trọng số cho mô hình
    model.apply(init_weights)

    # Tạo optimizer Adam để tối ưu hóa trọng số của mô hình
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Tạo lên lịch giảm tốc độ học
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.1, patience=2, verbose=False)

    # Xác định chỉ số của ký tự đặc biệt "pad" để được sử dụng trong hàm CrossEntropyLoss
    slot_pad_index = slot.vocab.stoi[text.pad_token]

    # Hàm mất mát cho slot và intent
    loss_slot = nn.CrossEntropyLoss(ignore_index=slot_pad_index)
    loss_intent = nn.CrossEntropyLoss()

    return model, optimizer, scheduler, loss_slot, loss_intent


In [ ]:
def train(model, iterator, optimizer, loss_slot, loss_intent, clip):
    # Chuyển mô hình sang chế độ huấn luyện
    model.train()

    # Khởi tạo các biến tổng lỗi
    epoch_loss = 0
    slot_epoch_loss = 0
    intent_epoch_loss = 0

    # Lặp qua từng batch trong iterator
    for i, batch in enumerate(iterator):
        # Trích xuất dữ liệu từ batch
        src, src_lens = batch.text  # Văn bản đầu vào và độ dài của nó
        trg, _ = batch.slot  # Chuỗi đích (slot)
        label = batch.intent  # Nhãn intent
        src = src.to(device)
        trg = trg.to(device)
        label = label.to(device)

        # Chạy mô hình để nhận dự đoán
        slot_outputs, intent_outputs = model(src, src_lens)

        # Reshape đầu ra và nhãn để tính toán lỗi
        output_dim = slot_outputs.shape[-1]
        slot_outputs = slot_outputs[:, 1:, :].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)

        # Tính lỗi cho từng phần của mô hình (slot và intent)
        loss1 = loss_slot(slot_outputs, trg)  # Lỗi cho slot
        loss2 = loss_intent(intent_outputs, label)  # Lỗi cho intent
        loss = loss1 + loss2  # Tổng lỗi

        # Lan truyền ngược và cập nhật trọng số
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)  # Ngăn chặn giá trị gradient quá lớn
        optimizer.step()
        optimizer.zero_grad()

        # Cập nhật tổng lỗi
        epoch_loss += float(loss.item())
        slot_epoch_loss += float(loss1.item())
        intent_epoch_loss += float(loss2.item())

    # Tính lỗi trung bình cho mỗi batch và trả về
    num_batches = len(iterator)
    return epoch_loss / num_batches, slot_epoch_loss / num_batches, intent_epoch_loss / num_batches


In [ ]:
def evaluate(model, iterator, loss_slot, loss_intent):
    # Chuyển mô hình sang chế độ đánh giá
    model.eval()

    # Khởi tạo các biến tổng lỗi
    epoch_loss = 0
    slot_epoch_loss = 0
    intent_epoch_loss = 0

    # Tính toán lỗi trên tập validation/test
    with torch.no_grad():
        # Lặp qua từng batch trong iterator
        for i, batch in enumerate(iterator):
            # Trích xuất dữ liệu từ batch
            src, src_len = batch.text  # Văn bản đầu vào và độ dài của nó
            trg, _ = batch.slot  # Chuỗi đích (slot)
            label = batch.intent  # Nhãn intent
            src = src.to(device)
            trg = trg.to(device)
            label = label.to(device)

            # Chạy mô hình để nhận dự đoán
            slot_outputs, intent_outputs = model(src, src_len)

            # Reshape đầu ra và nhãn để tính toán lỗi
            output_dim = slot_outputs.shape[-1]
            slot_outputs = slot_outputs[:, 1:, :].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            # Tính lỗi cho từng phần của mô hình (slot và intent)
            loss1 = loss_slot(slot_outputs, trg)  # Lỗi cho slot
            loss2 = loss_intent(intent_outputs, label)  # Lỗi cho intent
            loss = loss1 + loss2  # Tổng hợp lỗi

            # Cập nhật tổng lỗi
            epoch_loss += float(loss.item())
            slot_epoch_loss += float(loss1.item())
            intent_epoch_loss += float(loss2.item())

    # Tính lỗi trung bình cho mỗi batch và trả về
    num_batches = len(iterator)
    return epoch_loss / num_batches, slot_epoch_loss / num_batches, intent_epoch_loss / num_batches


In [ ]:
def train_model(model, train_iterator, val_iterator, optimizer, scheduler, loss_slot, loss_intent, n_epochs, clip, model_path, writer):
    # Khởi tạo biến lưu trữ lỗi tốt nhất trên tập validation
    best_valid_loss = float('inf')

    # Lặp qua số epoch đã định trước
    for epoch in range(n_epochs):
        # Bắt đầu tính thời gian để đoạn mã chạy mỗi epoch
        start_time = time.time()

        # Huấn luyện mô hình trên tập huấn luyện
        train_loss, train_slot_loss, train_intent_loss = train(model, train_iterator, optimizer, loss_slot, loss_intent, clip)

        # Ghi lỗi huấn luyện vào tensorboard
        writer.add_scalar('loss', train_loss, global_step=epoch+1)
        writer.add_scalar('slot_loss', train_slot_loss, global_step=epoch+1)
        writer.add_scalar('intent_loss', train_intent_loss, global_step=epoch+1)

        # Đánh giá mô hình trên tập validation
        valid_loss, valid_slot_loss, valid_intent_loss = evaluate(model, val_iterator, loss_slot, loss_intent)

        # Kết thúc tính thời gian cho epoch hiện tại
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        # Lưu mô hình nếu lỗi trên tập validation tốt hơn lỗi tốt nhất trước đó
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), model_path)

        # Điều chỉnh learning rate dựa trên lỗi tập validation
        scheduler.step(valid_loss)

        # In thông tin về quá trình huấn luyện và đánh giá
        print(f'epoch: {epoch + 1}, time-mins: {epoch_mins}, time-secs: {epoch_secs}')
        print(f'train loss: {train_loss}, train perplexity: {math.exp(train_loss)}')
        print(f'train slot loss: {train_slot_loss}, train slot perplexity: {math.exp(train_slot_loss)}')
        print(f'train intent loss: {train_intent_loss}, train intent perplexity: {math.exp(train_intent_loss)}')
        print(f'val loss: {valid_loss}, val perplexity: {math.exp(valid_loss)}')
        print(f'val slot loss: {valid_slot_loss}, val slot perplexity: {math.exp(valid_slot_loss)}')
        print(f'val intent loss: {valid_intent_loss}, val intent perplexity: {math.exp(valid_intent_loss)}')

    # Đóng writer TensorBoard
    writer.flush()
    writer.close()


In [ ]:
def epoch_time(start_time, end_time):
    # Tính thời gian chạy của một epoch
    run_tim = end_time - start_time
    # Chuyển đổi thời gian chạy thành phút và giây
    run_mins = int(run_tim / 60)
    run_secs = int(run_tim - (run_mins * 60))
    # Trả về thời gian chạy trong định dạng (phút, giây)
    return run_mins, run_secs

def init_weights(model):
    # Khởi tạo trọng số của mô hình với giá trị ngẫu nhiên trong khoảng [-0.5, 0.5]
    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.5, 0.5)


In [ ]:
# Khởi tạo đối tượng SummaryWriter để ghi log cho TensorBoard
writer = SummaryWriter(os.getcwd()+'/log', comment='intent_slot')

# Các tham số của mô hình và quá trình huấn luyện
text_emb_dim = 64
hidden_dim = 128
n_layers = 2
dropout = 0.2
lr = 0.005
gamma = 0.2
weight_decay = 0.2
n_epochs = 20
clip = 5.0
seq_len = 50

# Đường dẫn lưu trữ trọng số của mô hình
model_path = os.path.join(os.getcwd(), "model.h5")

# Xây dựng mô hình, tạo optimizer, scheduler và các hàm mất mát
model, optimizer, scheduler, loss_slot, loss_intent = build_model(TEXT,
                                                                  SLOT,
                                                                  INTENT,
                                                                  text_emb_dim,
                                                                  hidden_dim,
                                                                  n_layers,
                                                                  dropout,
                                                                  lr,
                                                                  gamma,
                                                                  weight_decay,
                                                                  seq_len)

# Không cần thiết lập AMP (Automatic Mixed Precision) vì đây là mã CPU

# Huấn luyện mô hình trên tập huấn luyện và đánh giá trên tập validation
train_model(model,
            train_iter,
            val_iter,
            optimizer,
            scheduler,
            loss_slot,
            loss_intent,
            n_epochs,
            clip,
            model_path,
            writer)


epoch: 1, time-mins: 0, time-secs: 7
train loss: 10.174558579266726, train perplexity: 26227.364185467773
train slot loss: 7.514621226669668, train slot perplexity: 1834.6724196850823
train intent loss: 2.659937266465072, train intent perplexity: 14.295392270056997
val loss: 2.888974905014038, val perplexity: 17.974874200729737
val slot loss: 0.23419204354286194, val slot perplexity: 1.2638871902765318
val intent loss: 2.654782772064209, val intent perplexity: 14.221896330572044
epoch: 2, time-mins: 0, time-secs: 7
train loss: 2.574801434527387, train perplexity: 13.12870999114768
train slot loss: 0.1594017527096874, train slot perplexity: 1.172809031250881
train intent loss: 2.4153996797708364, train intent perplexity: 11.194243576272893
val loss: 2.412687301635742, val perplexity: 11.163921695550659
val slot loss: 0.057294368743896484, val slot perplexity: 1.0589674914489544
val intent loss: 2.3553929328918457, val intent perplexity: 10.542270452774137
epoch: 3, time-mins: 0, time-se

In [ ]:
text_words_path = os.path.join(os.getcwd(), 'text_words.pkl')
with open(text_words_path, 'rb') as f_text_words:
    text_words = pickle.load(f_text_words)

slot_words_path = os.path.join(os.getcwd(), 'slot_words.pkl')
with open(slot_words_path, 'rb') as f_slot_words:
    slot_words = pickle.load(f_slot_words)


intent_words_path = os.path.join(os.getcwd(), 'intent_words.pkl')
with open(intent_words_path, 'rb') as f_intent_words:
    intent_words = pickle.load(f_intent_words)

In [ ]:
# Load the trained model
model.load_state_dict(torch.load(model_path))
model.eval()

# Evaluate the model on the test set
test_loss, test_slot_loss, test_intent_loss = evaluate(model, test_iter, loss_slot, loss_intent)

# Calculate perplexity
test_perplexity = math.exp(test_loss)
test_slot_perplexity = math.exp(test_slot_loss)
test_intent_perplexity = math.exp(test_intent_loss)

# Print or log the test evaluation results
print(f'Test loss: {test_loss}, Test perplexity: {test_perplexity}')
print(f'Test slot loss: {test_slot_loss}, Test slot perplexity: {test_slot_perplexity}')
print(f'Test intent loss: {test_intent_loss}, Test intent perplexity: {test_intent_perplexity}')


Test loss: 2.239734172821045, Test perplexity: 9.390834616541845
Test slot loss: 0.004253751132637262, Test slot perplexity: 1.004262811173818
Test intent loss: 2.235480308532715, Test intent perplexity: 9.350972125453064


In [ ]:
# Các thông số của mô hình
text_emb_dim = 64
hidden_dim = 128
n_layers = 2
dropout = 0.2
seq_len = 50

# Đường dẫn lưu trữ trọng số của mô hình
model_path = os.path.join(os.getcwd(), "model.h5")

# Số lượng từ trong từ điển của dữ liệu
input_dim = len(text_words)
output_dim = len(slot_words)
label_dim = len(intent_words)

# Khởi tạo mô hình
model = BirnnAttentionGate(input_dim, text_emb_dim, hidden_dim, n_layers, dropout, text_words['<pad>'], output_dim, label_dim, seq_len, True, True)

# Nạp trọng số đã được lưu
model.load_state_dict(torch.load(model_path))

# Chuyển mô hình về chế độ đánh giá
model.eval()

# Chuỗi đầu vào để dự đoán
sentence = "bật tv 4 ở 53 phần trăm"

# Dự đoán với đầu vào mới
with torch.no_grad():
    # Tokenize câu và thêm token kết thúc ('<eos>')
    tokenized = sentence.split()
    tokenized.append('<eos>')
    indexed = [text_words[t] for t in tokenized]

    # Chuyển đổi thành dạng Tensor
    tensor = torch.LongTensor(indexed)
    tensor = tensor.unsqueeze(0)

    # Dự đoán intent và slot
    slot_outputs, intent_outputs = model(tensor, [len(tensor[0])])
    intent = torch.argmax(intent_outputs, dim=1).item()
    slot_indices = torch.argmax(slot_outputs, dim=2)

    # Chuyển đổi các index slot thành chuỗi dự đoán, chú ý capitalize và thay thế token không mong muốn
    slot_prediction = [slot_words.itos[idx.item()].capitalize() if slot_words.itos[idx.item()] not in ['<eos>', '<sos>', '<unk>', '<pad>'] else 'O' for idx in slot_indices[0]]

    # In kết quả dự đoán
    print('slot_prediciton: {}'.format(' '.join(slot_prediction)))
    print('intent_prediction: {}'.format(intent_words.itos[intent]))


slot_prediciton: B-commandcommand B-devicedevice I-devicedevice O B-final-valuesyspercentage I-final-valuesyspercentage I-final-valuesyspercentage O
intent_prediction: smart.home.increase.level


In [ ]:
# Nạp trọng số của mô hình đã được huấn luyện
model.load_state_dict(torch.load(model_path))
model.eval()

# Đường dẫn đến file đầu vào chứa nhiều câu
input_file_path = "seq.in"
# Đường dẫn đến file kết quả đầu ra
output_file_path = "output_results.csv"

# Mở file đầu vào và file đầu ra để ghi kết quả
with open(input_file_path, 'r') as input_file, open(output_file_path, 'w', newline='') as output_file:
    # Sử dụng csv writer để viết vào file kết quả
    csv_writer = csv.writer(output_file)
    # Ghi dòng tiêu đề của file kết quả
    csv_writer.writerow(["text", "slot", "intent"])

    # Đọc từng câu từ file đầu vào
    for sentence in input_file.readlines():
        sentence = sentence.strip()

        # Dự đoán với mỗi câu
        with torch.no_grad():
            # Tokenize câu và thêm token kết thúc ('<eos>')
            tokenized = sentence.split()
            tokenized.append('<eos>')
            # Chuyển các từ thành index, sử dụng <unk> nếu từ không có trong từ điển
            indexed = [text_words.stoi[t] if t in text_words.stoi else text_words.stoi['<unk>'] for t in tokenized]
            tensor = torch.LongTensor(indexed)
            tensor = tensor.unsqueeze(0)

            # Dự đoán intent và slot
            slot_outputs, intent_outputs = model(tensor, [len(tensor[0])])
            intent = torch.argmax(intent_outputs, dim=1).item()
            slot_indices = torch.argmax(slot_outputs, dim=2)
            # Chuyển đổi các index slot thành chuỗi dự đoán, chú ý capitalize và thay thế token không mong muốn
            slot_prediction = [slot_words.itos[idx.item()].capitalize() if slot_words.itos[idx.item()] not in ['<eos>', '<sos>', '<unk>', '<pad>'] else 'O' for idx in slot_indices[0]]

            # Ghi kết quả vào file kết quả
            csv_writer.writerow([sentence, ' '.join(slot_prediction), intent_words.itos[intent]])

print(f"Results saved to {output_file_path}")


Results saved to output_results.csv


In [ ]:
df = pd.read_csv("output_results.csv")
df.head(20)

,text,slot,intent
0,tăng giúp mình mức độ của ốp tường thứ 1 lên với,O O O O O O B-devicedevice I-devicedevice I-de...,smart.home.increase.level
1,tôi muốn tăng đèn sắc màu thứ 3,O O O B-devicedevice I-devicedevice I-devicede...,smart.home.increase.level
2,tôi muốn tăng đèn cảnh 3,O O O B-devicedevice I-devicedevice I-devicede...,smart.home.increase.level
3,chòa bạn tôi muốn tăng mức độ của bóng ốp trần...,O O O O O O O O B-devicedevice I-devicedevice ...,smart.home.increase.level
4,giúp mình tăng đèn vườn thứ 1 lên 2 phần trăm ...,O O O B-devicedevice I-devicedevice I-devicede...,smart.home.increase.level
5,tăng hộ mình điện 4 lên 9 phần trăm nhé,O I-final-valuesyspercentage O B-devicedevice ...,smart.home.increase.level
6,chào bạn bạn tăng giúp mình bòng hắt trần thứ ...,O O O O O O B-devicedevice I-devicedevice I-de...,smart.home.increase.level
7,tăng tv lên 11 phần trăm bạn ơi,O B-devicedevice O I-devicedevice I-roomroom I...,smart.home.increase.level
8,tăng hộ mình bóng làm việc lên 11 phần trăm nhé,O O O B-devicedevice I-devicedevice I-devicede...,smart.home.increase.level
9,giúp mình tăng quạt thông gió 3 lên 5 phần tră...,O O O B-devicedevice I-devicedevice I-devicede...,smart.home.increase.level
